## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Загрузка и изучение данных

In [15]:
from pymystem3 import Mystem
from collections import Counter

import pandas as pd
df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В предоставленной статистике имеется информация по 21 525 кредитных запросам, сформированных по 12 признакам заявителя. Из них:
- У значений "Стаж работы" и "Суммарный доход" тип данных float;
- Также по репрезентативной выборе первых 5 строк видно, что содержатся ошибки непосредственно в самих заполнениях. Например, в колонке "Стаж работы" встречаются отрицательные значения;
- У всех заявителей указано количество детей и семейные статус, на основе которых будет строится анализ и ее влияние на кредитный скоринг.

### Шаг 2. Предобработка данных

Посмотрим повнимательнее на данные с пропусками, а конкретно - на тип занятости клиентов без дохода.

In [2]:
df[df['total_income'].isnull()].sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3609,0,NaN,64,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,жилье
2864,0,NaN,32,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции со своей недвижимостью
4139,0,NaN,38,высшее,0,Не женат / не замужем,4,M,компаньон,0,NaN,операции с недвижимостью
15818,0,NaN,26,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,операции с жильем
12904,0,NaN,53,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья


### Обработка пропусков

Выяснили, что имеются работающией заявители, у которых пропущены размеры заработной платы. Частично восстановим данные по имеющимся у нас значениям и заполним типичные числовых пропуски медианными значениями.

In [3]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
days_employed_avg = df['days_employed'].median()
total_income_avg = df['total_income'].median()

df['days_employed'] = df['days_employed'].fillna(days_employed_avg)
df['total_income'] = df['total_income'].fillna(total_income_avg)

In [5]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Имелось одинаковые количество пропусков (2 174) в столбцах "Стаж работы" и "Суммарный доход". Но, так как у заявителей указаны типы занятости, то скорее всего значения были просто упущены по ошибке. Для заполнения пропусков была вычислена медиана.

В столбце с данными "Количество детей" нет пропусков, но имеются следующие артефакты: невозможное количество детей "-1" у 47 заявителей (скорее всего ошиблись и проставили лишнее отрицательное значение), а также были замечены весьма невероятные семьи из 20 детей у 76 обращающихся за кредитом.

### Замена типа данных

Дробные части дней стажа и копейки в уровнях дохода нас не интересуют. А замена на целочисленный тип не только улучшает визуальное восприятие, но и сэкономит память и время.

In [6]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Изменены типы колонок "Стаж работы" и "Суммарный доход" методом astype в числовой тип данных для удобства последующей вычислительной работы. Приведены к нижнему регистру признаки "Семейный статус" и "Образование".

### Обработка дубликатов

In [7]:
df.duplicated().sum()

54

In [8]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [9]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [11]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [10]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

In [12]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2
df.duplicated().sum()

71

In [13]:
df = df.drop_duplicates()

In [14]:
df.duplicated().sum()

0

### Вывод

Первый подсчет количества дубликатов показал, что повторящихся строк было всего 54. Далее был проведен поиск по дополнительным вариантам дублей, а именно: 
- все символы в строках "Образование" были приведены к нижнему регистру, 
- числовые значения в столбце "Трудовой стаж" преобразованы из отрицательных чисел в положительные, 
- а также количество детей было поправлено с -1 на 1 и с 20 на 2. <br>

В результате, было найдено еще 17 дубликатов, то всего 71 повторяющихся строк, которые были удалены из таблицы методом drop_duplicates().

### Лемматизация

In [21]:
m = Mystem()

df['purpose_lemmas'] = df['purpose'].apply(m.lemmatize)
text = ' '.join(df.purpose)
lemmas_list = m.lemmatize(text)

Counter(lemmas_list)

Installing mystem to /Users/agermoge/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


Counter({'покупка': 5912,
         ' ': 55201,
         'жилье': 4473,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 858,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'получение': 1316,
         'высокий': 1375,
         'подержанный': 110,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612,
         '\n': 1})

### Вывод

В каждой строке были лемматизированы описания целей к их базовым словарным формам для последующей категоризации. Эти же леммы добавлены в новую строку lemmas_list для нахождения самых популярных тем, которые будут полезны для следующего шага "Категоризация данных".

### Категоризация данных

In [19]:
def children_group(children):
    if children == 0:
        return 'бездетные'
    elif children > 2:
        return 'многодетная семья'
    else:
        return 'стандартная семья'

def purpose_group(purpose):
    if 'недвижимость' in purpose:
        return 'недвижимость'
    elif 'жилье' in purpose:
        return 'недвижимость'
    elif 'строительство' in purpose:
        return 'недвижимость'
    elif 'подержанный' in purpose:
        return 'автомобиль'
    elif 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'свадьба' in purpose:
        return 'свадьба'
    elif 'сыграть' in purpose:
        return 'свадьба'
    elif 'образование' in purpose:
        return 'образование'
    else:
        return 'иные цели'

In [16]:
a = 0.35
b = 0.65
c = 0.90

df['total_income'].quantile(q=[a,b,c])  

0.35    119370.037656
0.65    175056.958875
0.90    278414.878581
Name: total_income, dtype: float64

In [17]:
def income_group(income):
    if income < 126000:
        return 'низкий доход'
    elif 270000 > income > 167000:
        return 'доход выше среднего'
    elif income > 270000:
        return 'высокий доход'
    else:
        return 'средний доход'

In [22]:
df['children_group'] = df['children'].apply(children_group)
df['purpose_group'] = df['purpose_lemmas'].apply(purpose_group)
df['income_group'] = df['total_income'].apply(income_group)

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,purpose_lemmas,purpose_group,income_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,стандартная семья,"[покупка, , жилье, \n]",недвижимость,доход выше среднего
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,стандартная семья,"[приобретение, , автомобиль, \n]",автомобиль,низкий доход
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,бездетные,"[покупка, , жилье, \n]",недвижимость,средний доход
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,многодетная семья,"[дополнительный, , образование, \n]",образование,доход выше среднего
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,бездетные,"[сыграть, , свадьба, \n]",свадьба,средний доход


### Вывод

Данные заявителей были категоризованы следующим образом: 
- По количеству детей: бездетные, стандартные семьи и многодетные (от 3х детей) для проверки гипотезы "влияет ли количество детей на процент возврата кредита";
- Также были выявление методом Counter и обобщены цели взятия заема на 4 самые популярные:  недвижимость, автомобиль, свадьба, образование и остальные были причислены к категории "Иные цели";
- Методом Quantile вычислены зарплатные вилки у каждой трети групп (0.35, 0.65, 0.9) для последующего распределения по группам: низкий доход, средний доход, доход выше среднего и высокий доход. 

### Шаг 3. Исследование вопросов

### 3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Найдем процент невозвратов: для этого количество просрочивших клиентов поделим на сумму всех клиентов в группе

In [11]:
df['debt1'] = df['debt']
relation_children = df.pivot_table(index=['children', 'children_group'], columns='debt', values='debt1', aggfunc='count')
relation_children['ratio'] = relation_children[1] / (relation_children[0] + relation_children[1]) * 100
print(relation_children)

debt                              0       1     ratio
children children_group                              
0        бездетные          13028.0  1063.0  7.543822
1        стандартная семья   4410.0   445.0  9.165808
2        стандартная семья   1926.0   202.0  9.492481
3        многодетная семья    303.0    27.0  8.181818
4        многодетная семья     37.0     4.0  9.756098
5        многодетная семья      9.0     NaN       NaN


На основе сводной таблицы можно сделать следующие выводы: 
- Зависимость возврата кредита от количества детей есть: кредиторы, у которых есть хотя бы 1 ребенок, погашают долги чаще (от 0,6% до 2%), чем бездетные;
- Самый высокий процент возврата у семьянинов с 4 детьми — 9.76%. Но стоит также отметить, что общая база данных заявителей мала (всего 37), так для анализа возврата кредита семьянина с 5 детьми данных недостаточно;
- Но при этом с учетом общего количества бездетных, которых практически в 3 раза больше семьянинов хотя бы с 1 ребенком, то 7,5% возвращающих кредит — это довольно хороший показатель для данной категории. 

### 3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
relation_family_status = df.pivot_table(index=['family_status'], columns='debt', values='debt1', aggfunc='count')
relation_family_status['ratio'] = relation_family_status[1] / (relation_family_status[0] + relation_family_status[1]) * 100
print(relation_family_status)

debt                       0    1     ratio
family_status                              
в разводе               1110   85  7.112971
вдовец / вдова           896   63  6.569343
гражданский брак        3763  388  9.347145
женат / замужем        11408  931  7.545182
не женат / не замужем   2536  274  9.750890


На основании полученных данных можно предположить:
- Люди, еще не вступившие в брак, с большей долей вероятности вернут кредит (9.7%). 
- Но чаще всего обращаются в банк семьянины: примерно 900 из 11 тыс погашают задолженности. 
- Самая пессимистичная статистика (6,6%) у людей, потерявших половинку, а также у тех, кто находится в разводе (7,1%).

### 3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [15]:
relation_income = df.pivot_table(index=['income_group'], columns='debt', values='debt1', aggfunc='count')
relation_income['ratio'] = relation_income[1] / (relation_income[0] + relation_income[1]) * 100
print(relation_income)

debt                    0    1     ratio
income_group                            
высокий доход        1990  151  7.052779
доход выше среднего  4943  420  7.831438
низкий доход         6848  617  8.265238
средний доход        5932  553  8.527371


Разница между группами с разным уровнем дохода необычна:
- Чаще всего выплачивают кредит люди с низким и средним доходом: средний показатель 8,4%
- Что интересно у группы с высоким доходом самый низкий уровень возврата (7%). Быть может это рок-н-рольщики, не вполне балансирующие своими растратами.

### 3.4. Как разные цели кредита влияют на его возврат в срок?

In [16]:
relation_goals = df.pivot_table(index=['purpose_group'], columns='debt', values='debt1', aggfunc='count')
relation_goals['ratio'] = relation_goals[1] / (relation_goals[0] + relation_goals[1]) * 100
print(relation_goals)

debt               0    1     ratio
purpose_group                      
автомобиль      3903  403  9.359034
недвижимость   10029  782  7.233373
образование     3643  370  9.220035
свадьба         2138  186  8.003442


Возвращают, по всей видимости, люди, берущие кредит на более ликвидные, а также менее дорогостоящие цели — "автомобиль" (9,3%) и "образование" (9,2%). Чаще всего обращаются за кредитом с целями, связанными с недвижимостью и при этом процент возврата в данном случае самый низкий (7,2%).

### Шаг 4. Общий вывод

В результате анализа были сделаны следующие выводы:
- Самый высокий процент возврата у следующий категорий: не вступивших в брак (9.7%) или проживающих в гражданском браке лиц (9.3%), а также у семьянинов (от 8 до 9.7%). 
- При этом чаще всего возвращают кредит (˜8,4%) заявители с низким и средним уровнем дохода, а также лица, запрашивающие кредит на сравнительно недорогие цели, такие как: "автомобиль" (9,3%) и "образование" (9,2%).
- Примерно одинаково погшают долги следующие категории лиц: бездетные (7.5%), семейные (7.5%), а также с целью "Недвижимость" (7.2%). В то же самое время это самые крупные базы — от 10 до 13 тыс людей в каждой группе.

При анализе кредитного скоринга, по моему мнению стоит также брать во внимание общее количество заявителей определенной группы. Так, например в разы больше обращаются за кредитом категория бездетных и лиц, чьи цели связанны с недвижимостью. Имеется гипотеза, что даже при наличии не самых высоких процентов возврата у данных групп, в случае одобрения кредита у банка будет бОльший итоговый возврат с учетом общего количества заявителей. В случае подтверждения гипотезы, при расчета скоринга стоит учесть такой критерий как популярность категории.